# Język Python - Wykład 3

## Closures (Domknięcia)

#### Domknięcie - funkcja wraz z otoczeniem referencyjnym (wszystkie zmienne z czasu utworzenia) - dostępne nawet po opuszczeniu otoczenia leksykalnego

Funkcje zagnieżdzone:

In [1]:
def outer():
    x = 137
    def inner():
        print(x)
    inner()

In [2]:
outer()

137


Funkcje jako obiekty

In [3]:
def add(x,y):
    return x+y
def apply(f,x,y):
    return(f(x,y))

In [4]:
apply(add,2,2)

4

Domknięcia

In [5]:
def generate_power_func(n):
    print("id(n): %X" % id(n))
    def nth_power(x):
        return x**n
    print("id(nth_power): %X" % id(nth_power))
    return nth_power

In [6]:
raised_to_4 = generate_power_func(4)

id(n): 7F35CAF33700
id(nth_power): 7F35B9B77EA0


In [7]:
repr(raised_to_4)

'<function generate_power_func.<locals>.nth_power at 0x7f35b9b77ea0>'

In [8]:
del generate_power_func

In [9]:
generate_power_func(1)

NameError: name 'generate_power_func' is not defined

In [13]:
raised_to_4(3)

81

In [14]:
raised_to_4.__closure__[0].cell_contents

4

Domkniecia c.d.

In [12]:
def outer():
    x = 137
    def inner():
        nonlocal x
        print("inner",x)
        x = 0
    print("outer pre",x)
    inner()
    print("outer post",x)

In [13]:
outer()

outer pre 137
inner 137
outer post 0


### Domknięcie w Groovy

    def localMethod() {
      def localVariable = new java.util.Date()
      return { println localVariable }
    }
    def clos = localMethod()


### Domknięcie w JavaScript

    function createMarker(point, number) {
        var marker = new GMarker(point);
        var message = ["This","is","the","secret","message"];
        marker.value = number;
        GEvent.addListener(marker, "click", function() {
            var myHtml = "<b>#" + number + "</b><br/>" +
                message[number -1];
            map.openInfoWindowHtml(point, myHtml);
        });
        return marker;
    }


## Dekoratory

In [14]:
def enhanced(function):
    def tmp(y):
        print("But now, I am enhanced")
        return function(y)
    return tmp


In [15]:
def f(n):
    print("Just a simple method printing %d." % n)

In [16]:
f(1)

Just a simple method printing 1.


In [17]:
f = enhanced(f)

In [18]:
f(1)

But now, I am enhanced
Just a simple method printing 1.


In [19]:
@enhanced
def g(n):
    print("Another simple method printing %d." % n)

In [20]:
g(2)

But now, I am enhanced
Another simple method printing 2.


In [21]:
def veryenhanced(function):
    def new(y):
        print("But now, I am very enhanced")
        return function(y)
    return new

In [22]:
@veryenhanced
@enhanced
def h(n):
    print("So simple method printing %d." % n)

In [23]:
h(3)

But now, I am very enhanced
But now, I am enhanced
So simple method printing 3.


    #Example ...
    @synchronized
    @logging
    def myfunc(arg1, arg2, ...):
    # ...do something


Ciąg dalszy nastąpi za chwilę...

## Obiektowość

In [25]:
class MyClass:
    
    def f(self,m):
        self.n = m
        return 'hello world'
    
    def __init__(self):
        self.n = 12345  #pole zwykłe
    i = 12345  #pole statyczne

m = MyClass()
m.f(1)
    

'hello world'

In [26]:
print(MyClass.i)

12345


In [27]:
m2 = MyClass()
print(m2.i)

12345


In [28]:
#podstawienie
m.i -= 1
print(m.i, MyClass.i, m2.i)

12344 12345 12345


In [29]:
class MyClass(object):
    
    def method1(self, x):
        print(x)
    
    @staticmethod
    def somestaticmethod(x):
        """
        metoda statyczna nie przyjmuje argumentu 'self'- 
        nie ma dostępu do atrybutów klasy/instancji.
        """
        print(x)
            
m = MyClass()
MyClass.somestaticmethod(1)
print()
m.somestaticmethod(2)

f = MyClass.somestaticmethod
f(4)
print(f.__class__)

1

2
4
<class 'function'>


In [30]:
class MyClass(object):
    
    def method1(self, x):
        print(x)
    
    @classmethod
    def someclassmethod(cls, x):
        """
        w metodzie klasy, do atrybutu __self__ funkcji,
        przypisywana jest klasa a nie instancja (jak w przypadku tradycyjnych metod)
        """
        print(cls)
        print(x)

        
m = MyClass()
MyClass.someclassmethod(1)
print
m.someclassmethod(2)

g = MyClass.someclassmethod
print(g.__class__)

<class '__main__.MyClass'>
1
<class '__main__.MyClass'>
2
<class 'method'>


In [31]:
class Employee:
    pass


john = Employee()  # Create an empty employee
# Fill the fields of the record
john.name = 'John Doe'
john.dept = 'computer lab'
john.salary = 1000

### Nie ma pól i metod prywatnych, ale...

### Pola zaczynające się od „__” mają nazwę zmienioną

In [32]:
class MyClass:
    
    def __init__(self):
        self.__n = 12345  #pole "prywatne"

m = MyClass()
print(dir(m))
print(m._MyClass__n)

['_MyClass__n', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']
12345


## Metody Specjalne (wprowadzenie)

    __init__ - konstruktor
    __del__ - destruktor (nie korzystamy - bo garbage collector jest nieprzewidywalny)
    __add__, __sub__, __mul__ etc. - przeciążanie operatorów
    __lt__ (<), __gt__ (<=), __eq__ (==), __ne__ (!=,<>), etc ... porównanie
    __call__ - obiekt staje się funktorem - można go wywołać jak funkcję

In [33]:
class MyClass(object):
    """ To jest docstring """
    
    def __init__(self):
        self.n=1
        
    def __getattr__(self, name):
        print("Looking for %s" % name)
        return 0
  

In [34]:
a = MyClass()

print(a.n)

print()

print(a.m)

1

Looking for m
0


In [35]:
print(MyClass.__dict__)

print()

print(a.__dict__)

{'__dict__': <attribute '__dict__' of 'MyClass' objects>, '__module__': '__main__', '__doc__': ' To jest docstring ', '__getattr__': <function MyClass.__getattr__ at 0x7f30fc0f8620>, '__init__': <function MyClass.__init__ at 0x7f30fc0f8598>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>}

{'n': 1}


In [36]:
class Borg:

    __shared_state = {}

    def __init__(self):
        self.__dict__ = self.__shared_state
        
a = Borg()
b = Borg()

print(a is b)
print()

a.n = 17
print(b.n)

False

17


In [37]:
class MyClass(object):
    """ To jest doctring """
    
    def __init__(self):
        self.n = 1
        
    def __getattribute__(self, name):
        print("Looking for %s" % name)
        return 0
      
    def __setattr__(self, name, value):
        print("Setting %s at %s" % (name, value)) #ta funkcja kłamie, to pole wcale nie ma takiej wartości
        return 0
    

In [38]:
a = MyClass()

print(a.n)
print(a.m)

print()

a.n = 3
a.m = 2

Setting n at 1
Looking for n
0
Looking for m
0

Setting n at 3
Setting m at 2


In [39]:
class C(object):
    def __init__(self):
        self._x = 3

    @property
    def x(self):
        """I'm the 'x' property."""
        return self._x

    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x

In [40]:
c = C()

print(c.x)

3


## Dekoratory, c.d.

In [41]:
def entryExit(f):
    def new_f():
        print("Entering", f.__name__)
        f()
        print("Exited", f.__name__)
    return new_f


In [42]:
@entryExit
def func1():
    print("inside func1()")
func1()

Entering func1
inside func1()
Exited func1


In [43]:
class entryExit(object):
    
    def __init__(self, f):
        self.f = f
        self.n = 0
    
    def __call__(self):
        self.n += 1
        print("Entering", self.f.__name__, self.n, "time" + ("s" if self.n > 1 else ""))
        self.f()
        print("Exited", self.f.__name__)

        
@entryExit
def func1():
    print("inside func1()")


In [44]:
type(func1)

__main__.entryExit

In [56]:
func1()

Entering func1 12 times
inside func1()
Exited func1


In [57]:
class Decorator(object):
    
    def __init__(self, arg):
        self.arg = arg
    
    def __call__(self, cls):
        class Wrapped(cls):
            classattr = self.arg
            def new_method(self, value):
                return value * 2
        return Wrapped

In [59]:
@Decorator("decorated class")
class TestClass(object):
    
    def new_method(self, value):
        return value * 3
    
t = TestClass()

print(t.new_method(5))

10


In [60]:
t.classattr

'decorated class'